In [2]:
!pip install numpy pandas matplotlib plotly gensim scikit-learn nltk


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import gensim
import nltk
from sklearn.feature_extraction.text import CountVectorizer


In [5]:
import pandas as pd

# Load the first dataset (replace with your dataset path)
data = pd.read_csv(r'C:\Users\b0161166\Downloads\MovieReviews\letterboxd-reviews.csv', encoding='ISO-8859-1')

# Load the second dataset (replace with your dataset path)
data2 = pd.read_csv(r'C:\Users\b0161166\Downloads\MovieReviews\metacritic-reviews.csv', encoding='ISO-8859-1', on_bad_lines='skip')

# Display the first few rows of both datasets to confirm they've loaded correctly
print(data.head())
print(data2.head())


                                 Movie name Release Year           Rating  \
0                           Aftersun (2022)         2022   â??â??â??â??Â½   
1                              Joker (2019)         2019  â??â??â??â??â??   
2       Puss in Boots: The Last Wish (2022)         2022            â??Â½   
3          The Banshees of Inisherin (2022)         2022  â??â??â??â??â??   
4  Everything Everywhere All at Once (2022)         2022         â??â??Â½   

     Reviewer name Review date  \
0           Tuomas   12-Jan-20   
1             Joao   20-Dec-22   
2         NicoPico   15-Sep-22   
3        Ella Kemp    8-Apr-22   
4  CosmonautMarkie   14-Aug-19   

                                              Review Comment count  \
0                  This review may contain spoilers.           130   
1  if youâ??ve never swam in the ocean then of co...          1.8K   
2                Puss in Boots: Into the Pussy-Verse          6  2   
3  I will NOT leave my donkey outside when Iâ??m sad

In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download required NLTK resources (run this only once)
nltk.download('stopwords')
nltk.download('wordnet')

# Define the preprocessing function
def preprocess_text(text):
    # Check if text is a valid string
    if isinstance(text, str):
        stop_words = set(stopwords.words('english'))  # Get English stopwords
        lemmatizer = WordNetLemmatizer()  # Initialize lemmatizer
        tokens = text.split()  # Split text into words
        # Remove stopwords and lemmatize
        tokens = [lemmatizer.lemmatize(word.lower()) for word in tokens if word.lower() not in stop_words]
        return " ".join(tokens)
    else:
        # If text is not valid, return an empty string
        return ""

# Apply the preprocessing function to the 'Review' column in the first dataset
data['cleaned_text'] = data['Review'].apply(preprocess_text)

# Apply the preprocessing function to the 'summary' column in the second dataset
data2['cleaned_text'] = data2['summary'].apply(preprocess_text)

# Display the first few rows of the cleaned text from both datasets
print("First dataset cleaned text:")
print(data[['Review', 'cleaned_text']].head())

print("\nSecond dataset cleaned text:")
print(data2[['summary', 'cleaned_text']].head())


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\b0161166\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\b0161166\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


First dataset cleaned text:
                                              Review  \
0                  This review may contain spoilers.   
1  if youâ??ve never swam in the ocean then of co...   
2                Puss in Boots: Into the Pussy-Verse   
3  I will NOT leave my donkey outside when Iâ??m sad   
4  Watch it and have fun before film Twitter tell...   

                                        cleaned_text  
0                       review may contain spoilers.  
1  youâ??ve never swam ocean course pool seems deep.  
2                             pus boots: pussy-verse  
3                     leave donkey outside iâ??m sad  
4       watch fun film twitter tell itâ??s overrated  

Second dataset cleaned text:
                                             summary  \
0  This film noir portrait of corruption and mora...   
1  Seven Samurai (Shichinin no samurai) tells the...   
2  An aging group of outlaws look for one last bi...   
3  Robert Bresson's 1966 film focuses on the stor..

In [37]:
import gensim
from gensim import corpora
from gensim.models import LdaModel

# Step 1: Define the function to apply LDA and display topics in descending order of frequency
def apply_lda_and_display_topics(data, num_topics_list, dataset_name="Dataset"):
    # Tokenize the cleaned text
    texts = [text.split() for text in data['cleaned_text']]

    # Create a dictionary and corpus for the dataset
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]

    # Loop through each value of N (number of topics)
    for num_topics in num_topics_list:
        lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)
        
        print(f"\nTopics for {dataset_name} (N={num_topics}):")
        
        # Get the topics sorted by frequency (descending order)
        topic_list = lda_model.print_topics(num_words=5)
        
        # Calculate frequency of each topic
        topic_frequency = {}
        for doc in corpus:
            topics = lda_model.get_document_topics(doc)
            for topic_id, topic_prob in topics:
                if topic_id not in topic_frequency:
                    topic_frequency[topic_id] = topic_prob
                else:
                    topic_frequency[topic_id] += topic_prob

        # Sort topics by frequency in descending order
        sorted_topic_frequency = sorted(topic_frequency.items(), key=lambda x: x[1], reverse=True)
        
        # Display the sorted topics in descending order of frequency
        for topic_id, freq in sorted_topic_frequency:
            print(f"Topic {topic_id+1}: {lda_model.print_topic(topic_id)} - Frequency: {freq}")

# Step 2: Apply LDA to both datasets
num_topics_list = [5, 10, 20, 50]

# Apply LDA and display topics for the first dataset
apply_lda_and_display_topics(data, num_topics_list, dataset_name="Dataset 1")

# Apply LDA and display topics for the second dataset
apply_lda_and_display_topics(data2, num_topics_list, dataset_name="Dataset 2")



Topics for Dataset 1 (N=5):
Topic 4: 0.025*"movie" + 0.016*"â" + 0.010*"like" + 0.005*"time" + 0.005*"one" + 0.005*"get" + 0.005*"best" + 0.004*"people" + 0.004*"give" + 0.004*"love" - Frequency: 1132.8553466796875
Topic 3: 0.010*"movie" + 0.008*"like" + 0.008*"ever" + 0.007*"film" + 0.006*"iâ??m" + 0.006*"one" + 0.006*"fuck" + 0.005*"think" + 0.005*"watch" + 0.004*"want" - Frequency: 1081.7041015625
Topic 5: 0.011*"movie" + 0.011*"like" + 0.008*"good" + 0.008*"film" + 0.005*"really" + 0.005*"every" + 0.005*"actually" + 0.005*"one" + 0.005*"make" + 0.004*"know" - Frequency: 1003.79541015625
Topic 1: 0.010*"like" + 0.006*"movie" + 0.005*"gay" + 0.005*"know" + 0.005*"want" + 0.005*"get" + 0.004*"one" + 0.004*"song" + 0.003*"look" + 0.003*"really" - Frequency: 858.4805297851562
Topic 2: 0.093*"?â" + 0.003*"birthday" + 0.003*"happened" + 0.003*"gonna" + 0.002*"decided" + 0.002*"woof" + 0.002*"2" + 0.002*"full" + 0.002*"suck" + 0.002*"?" - Frequency: 512.2046508789062

Topics for Dataset 1

In [7]:
import plotly.express as px
import plotly.offline as pyo
import plotly.io as pio
import gensim
from gensim import corpora
from gensim.models import LdaModel

# Initialize Plotly in notebook mode
pyo.init_notebook_mode(connected=True)

# Define the function to apply LDA and plot donut charts
def apply_lda_and_plot_donut(data, num_topics_list, dataset_name="Dataset"):
    # Tokenize the cleaned text
    texts = [text.split() for text in data['cleaned_text']]

    # Create a dictionary and corpus for the dataset
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]

    # Loop through each value of N (number of topics)
    for num_topics in num_topics_list:
        lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

        # Get the topic frequencies
        topic_frequency = {i: 0 for i in range(num_topics)}
        for doc in corpus:
            topics = lda_model.get_document_topics(doc)
            for topic_id, topic_prob in topics:
                topic_frequency[topic_id] += topic_prob
        
        # Prepare data for plotting
        topic_labels = [f"Topic {i+1}" for i in range(num_topics)]
        topic_counts = [topic_frequency[i] for i in range(num_topics)]

        # Plot donut chart for the current dataset and number of topics
        fig = px.pie(names=topic_labels, values=topic_counts, hole=0.3, title=f"Topic Distribution in {dataset_name} (N={num_topics})")
        
        # Ensure plot is displayed using plotly.io.show()
        pio.show(fig)

# Step 2: Apply LDA to both datasets for 5, 10, 20 topics
num_topics_list = [5, 10, 20]

# Apply LDA and plot donut charts for the first dataset
apply_lda_and_plot_donut(data, num_topics_list, dataset_name="Dataset 1")

# Apply LDA and plot donut charts for the second dataset
apply_lda_and_plot_donut(data2, num_topics_list, dataset_name="Dataset 2")


In [10]:
# Save the figures as HTML files with specific paths for each dataset and number of topics
def save_figures_as_html(fig, num_topics, dataset_name, output_dir="C:/Users/b0161166/Downloads/"):
    # Create a unique filename based on the dataset name and number of topics
    output_file = f"{output_dir}{dataset_name}_topic_distribution_N{num_topics}.html"
    fig.write_html(output_file)
    print(f"Saved {output_file}")

# Inside the apply_lda_and_plot_donut function, we will save each figure as HTML:
def apply_lda_and_plot_donut(data, num_topics_list, dataset_name="Dataset", output_dir="C:/Users/b0161166/Downloads/"):
    # Tokenize the cleaned text
    texts = [text.split() for text in data['cleaned_text']]

    # Create a dictionary and corpus for the dataset
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]

    # Loop through each value of N (number of topics)
    for num_topics in num_topics_list:
        lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

        # Get the topic frequencies
        topic_frequency = {i: 0 for i in range(num_topics)}
        for doc in corpus:
            topics = lda_model.get_document_topics(doc)
            for topic_id, topic_prob in topics:
                topic_frequency[topic_id] += topic_prob

        # Prepare data for plotting
        topic_labels = [f"Topic {i+1}" for i in range(num_topics)]
        topic_counts = [topic_frequency[i] for i in range(num_topics)]

        # Plot donut chart for the current dataset and number of topics
        fig = px.pie(names=topic_labels, values=topic_counts, hole=0.3, title=f"Topic Distribution in {dataset_name} (N={num_topics})")
        
        # Save the figure as an HTML file using the helper function
        save_figures_as_html(fig, num_topics, dataset_name, output_dir)

# Apply LDA to both datasets for 5, 10, 20 topics
num_topics_list = [5, 10, 20]

# Apply LDA and plot donut charts for the first dataset and save HTML files
apply_lda_and_plot_donut(data, num_topics_list, dataset_name="Dataset_1")

# Apply LDA and plot donut charts for the second dataset and save HTML files
apply_lda_and_plot_donut(data2, num_topics_list, dataset_name="Dataset_2")


Saved C:/Users/b0161166/Downloads/Dataset_1_topic_distribution_N5.html
Saved C:/Users/b0161166/Downloads/Dataset_1_topic_distribution_N10.html
Saved C:/Users/b0161166/Downloads/Dataset_1_topic_distribution_N20.html
Saved C:/Users/b0161166/Downloads/Dataset_2_topic_distribution_N5.html
Saved C:/Users/b0161166/Downloads/Dataset_2_topic_distribution_N10.html
Saved C:/Users/b0161166/Downloads/Dataset_2_topic_distribution_N20.html
